# Banana Navigation Project

![Screenshot of banana environment](doc/bannerImage.png)

This is an implementation of Deep Reinforcement Q-Learning, applied to train an agent with four possible actions (move left, right, forward, or backward), to pick up yellow bananas and avoid blue bananas.

## Table of Contents
+ Environment Setup
+ Description of Algorithm
  - Value Distribution
  - Parameterized Model
  - Prioritized Replay
+ Implementation of Algorithm
  - Hyperparameters
  - Prioritized Replay Buffer
  - Action Value Distribution Function (Neural Network)
  - Bellman Update (Loss) Computation
  - Training Loop
+ Training
+ Results
+ References

## Environment Setup

+ Follow instructions [here](https://github.com/udacity/Value-based-methods#dependencies) to set up the environment, *with the following changes:*
  - Before running `pip install .`, edit `Value-based-methods/python/requirements.txt` and remove the `torch==0.4.0` line
  - After running `pip install .`, run the appropriate PyTorch installation command for your system indicated [here](https://pytorch.org/get-started/locally/)
  - Continue following the instructions [here](https://github.com/udacity/Value-based-methods#dependencies) to their conclusion.
+ Download the appropriate Unity Environment for your platform:
  - [Linux](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P1/Banana/Banana_Linux.zip)
  - [Mac OSX](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P1/Banana/Banana.app.zip)
  - [Windows (32-bit)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P1/Banana/Banana_Windows_x86.zip)
  - [Windows (64-bit)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P1/Banana/Banana_Windows_x86_64.zip)
+ Place the Unity Environment zip file in the `p1_navigation/` folder of the repository cloned in the first step, and unzip the file.

### Supplemental Packages
Run the following code cell *once* to install additional packages required by the implementation

### Imports and references
Run the following code cell at every kernel instance start-up to bring implementation dependencies into the notebook namespace, and identify the path to the simulated environment executable.

In [70]:
from unityagents import UnityEnvironment
from collections import namedtuple, deque
import numpy as np
import torch
import torch.nn as nn

# Set to the path to simulated environment executable on system.
env_location = \
"C:\Projects\Value-based-methods\p1_navigation\Banana_Windows_x86_64\Banana_Windows_x86_64\Banana.exe"

## Description of Algorithm

Deep Reinforcement Q-Learning is a *value-based* class of reinforcement learning algorithms.  These algorithms aim to accurately approximate either the expected reward or reward probability distribution, for every possible pair (state, agent response) in the environment.  With either of these approximations, an agent may be controlled by, when in each state, selecting the action with the highest expected reward.

### Value Distribution
This implementation, like in aims to find the reward probability distribution [1]:<br><br>
$$d_t^{(n)}\equiv(R_t^{(n)}+\gamma_t^{(n)}\textbf{z},\textbf{p}(S_{t+n},a^{*}_{t+n}))$$
<br>
This is an *n-step* value distribution.  The value of the random variable $d_t^{(n)}$ is the sum $R_t^{(n)}$ of the rewards over the next *n* environment time steps, plus the reward distribution $\textbf{z}$ discounted by the factor $\gamma_t^{(n)}$.  The probabilities for the values for the random variable are those that result from, when the agent is in the state $S_{t+n}$, *n* steps advanced from present, the optimal action $a^{*}_{t+n}$ is selected. <br><br>
In practice, the continuous distribution of values is approximated by histogram binning.  The bins are called *atoms* in the literature and typically form an evenly spaced grid between maximum and minimum allowed values $v_{max}$ and $v_{min}$.

### Parameterized Model
As the product of the state and action spaces is very large (infinite, since the state variables are continuous), it is necessary to represent the reward distribution with a parameterized function.  The 'Deep' in Deep Reinforcement Q-Learning implies that the parameterized function is going to be a multi-layer neural network.

Using the notation in [1], let $p_{\theta}^i(s,a)$ denote this function, with set of parameters $\theta$.  Optimization of the parameters in $\theta$ shall be performed, such that, given the selection of an action $a$ by the agent, when the environment is in state $s$, $p_{\theta}^i(s,a)$ approximates the probability that the *n-step* reward will be $z_i$.  As in [1], the available $z_i$ will be defined by:<BR><BR>
$$z_i \equiv v_{min} + (i-1)\frac{v_{max}-v_{min}}{N_{atoms}-1},  i \in {1,...,N_{atoms}}$$
<BR>

The neural network will be a fully connected MLP with one input for each state variable, at least one hidden layer, and one output for each pair $(z_i, a)$.  See the implentation section for details.

### Prioritized Replay
The algorithm will periodically switch between exploration and learning phases.  <br><br>During exploration phases, state transition tuples $(S_t,a_t,r_t,S_{t+1})$ will be collected, transformed to *n-step* transition events via an accumulation buffer, and stored in a prioritized experience buffer. 
<br><br>
During learning phases, transition events sampled from the prioritized experience buffer will be used to optimize the parameterized model.  Like in [1], the probability of utilizing a transition $T$ from the experience buffer is consistent with the proportionality relation: <br><br>
$$p_T \varpropto (Loss)^{\omega}, \omega \in [0,\infty)$$
<br>
The hyperparameter $\omega$ allows tuning of the degree to which the probability of selection is affected by loss magnitude [3].
<br><br>Qualitatively, the $Loss$ in this context is proportional to how inconsistent the parameterized model's prediction is with a prediction that uses actual rewards sampled from the environment.  See the implementation section for detail on how the loss is computed.

## Implementation of Algorithm

### Hyperparameters

#### Environment
`state_dimension`: Dimension of the observable state space<br>
`num_actions`: Number of actions available to agent

In [68]:
class EnvironmentHyperparameters():
    def __init__(self,state_dimension=37,num_actions=4):
        self.state_dimension = state_dimension
        self.num_actions = num_actions

#### Parameterized Model
`num_hidden_layers`: How many hidden layers are included in the neural network model<br>
`hidden_layer_size`: How many neurpns are included in each hidden layer

In [4]:
class ModelHyperparameters():
    def __init__(self,num_hidden_layers=1,hidden_layer_size=200):
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layer_size = hidden_layer_size

#### Reward Distribution
`v_min`: Lower limit clipping value for *n-step* return<br>
`v_max`: Upper limit clipping value for *n-step* return<br>
`gamma`: Discount factor for reward calculation.  An expected reward n steps in the future is discounted by $\gamma^n$<br>
`n_step_order`: How many environment steps from initial state are considered in constructing action value distribution<br>
`n_atoms`: Number of discretization bins for representing possible value distribution returns

In [6]:
class DistributionHyperparameters():
    def __init__(self,v_min=-5,v_max=5,gamma=1,n_step_order=10,n_atoms=51):
        self.v_min = v_min
        self.v_max = v_max
        self.gamma = gamma
        self.n_step_order = n_step_order
        self.n_atoms = n_atoms

#### Replay Buffer
`buffer_size`: Maximum number of transition events that may be stored in the replay buffer<br>
`omega`: Loss influence factor for transition event selection probabilities

In [21]:
class ReplayBufferHyperparameters():
    def __init__(self,buffer_size=1e6,omega=0.5):
        self.buffer_size = buffer_size
        self.omega = omega

#### Optimization Behavior
`lr`: Learning rate for parameterized model optimizer<br>
`learn_interval`: Number of environment exploration steps between each learning phase<br>
`batch_size`: Number of n-step transition events to process during each learning phase <br>
`target_alpha`: Soft update factor used so that target reward distribution lags policy reward distribution.  See [this explanation](https://deeplizard.com/learn/video/xVkPh9E9GfE) for why this is necessary.  The following update will be applied once per batch:<br><br>
$$\theta_{target} = (1-\alpha)\theta_{target} + \alpha\theta_{policy}$$ <br>


In [23]:
class OptimizationHyperparameters():
    def __init__(self,lr=1e-4,learn_interval=50,batch_size=32,target_alpha=1e-2):
        self.lr = lr
        self.learn_interval = learn_interval
        self.batch_size = batch_size
        self.target_alpha = target_alpha

### Prioritized Replay Buffer

Like in [3], a sum tree structure is used for efficient weighted sampling over a large number of items.  This implementation is heavily based on that in reference [4](http://www.sefidian.com/2022/09/09/sumtree-data-structure-for-prioritized-experience-replay-per-explained-with-python-code/), but with some organizational changes to accomodate dynamic resizing.

In [44]:
# Object that represents an experience in the experience buffer and/or a non-leaf node of the sum tree
# Experience tuples are stored in the self.data attribute
class SumTreeNode():
    
    def __init__(self,data=None,p_i=0):
        self.data = data
        self.p_i = p_i
        self.parent = None
        self.left_child = None
        self.right_child = None
    
    def update_p(self, delta_p):
        self.p_i += delta_p
        if self.parent is not None:
            self.parent.update_p(delta_p)
    
    def attach_child(self,child):
        if self.data is None:    # Not a leaf node
            if self.left_child is None:    # No children, become leaf with cloned data
                self.data = child.data
                self.update_p(child.p_i - self.p_i)
            else:    # Non-leaf node, attach to lower p_i side
                if self.left_child.p_i < self.right_child.p_i:
                    delegate_node = self.left_child
                else:
                    delegate_node = self.right_child
                delegate_node.attach_child(child)
        else:    # self is a leaf-node.  Clone self.data into new child, become non-leaf
            self.left_child = SumTreeNode(self.data,self.p_i)
            self.data = None
            self.right_child = child
            self.left_child.parent, self.right_child.parent = self, self     
            self.update_p((self.left_child.p_i + self.right_child.p_i)- self.p_i)
    
    def remove(self):
        if self.parent is not None:  # remove only has effect if there is a parent
            if self.parent.left_child is self:
                sibling = self.parent.right_child
            else:
                sibling = self.parent.left_child
            # Parent will become a leaf, clone data from sibling and update references
            self.parent.data = sibling.data
            self.parent.update_p(sibling.p_i - self.parent.p_i)
            self.parent.left_child, self.parent.right_child = None, None
    
    def weighted_retrieve(self,p_samp):
        if self.data is not None: # must be a leaf-node
            return self
        else:
            if self.left_child.p_i >= p_samp:
                return self.left_child.weighted_retrieve(p_samp)
            else:
                return self.right_child.weighted_retrieve(p_samp - self.left_child.p_i)
            
    # This is used to identify low sampling probability nodes for removal, and determine
    # the lowest p_i, which is used to normalize the magnitude of parameterized model updates
    def minimal_node(self):
        if self.data is not None:
            return self
        else:
            min_left = self.left_child.minimal_node()
            min_right = self.right_child.minimal_node()
            return min_left if min_left.p_i < min_right.p_i else min_right
        
    def describe(self):
        desc_str = f'{"Non-leaf" if self.data is None else str(self.data)}: {str(self.p_i)}'
        if self.left_child is not None:
            left_desc_str = \
                f'{"Non-leaf" if self.left_child.data is not None else str(self.left_child.data)}' \
                +f': {self.left_child.p_i}'
        else:
            left_desc_str = "Nothing"
        if self.right_child is not None:
            right_desc_str = \
                f'{"Non-leaf" if self.right_child.data is not None else str(self.right_child.data)}' \
                +f': {self.right_child.p_i}'
        else:
            right_desc_str = "Nothing"
        print(f'{desc_str} --> {left_desc_str} | {right_desc_str}')
        if self.left_child is not None:
            self.left_child.describe()
        if self.right_child is not None:
            self.right_child.describe()       

In [45]:
# Prioritized replay buffer definition

# Experience aggregate
Experience = namedtuple('Experience',['state','action','reward','last_state'])

class PrioritizedReplayBuffer():
    
    def __init__(self,replay_buffer_hyperparameters=ReplayBufferHyperparameters()):
        self.max_size = replay_buffer_hyperparameters.buffer_size
        self.omega = replay_buffer_hyperparameters.omega
        self.store = SumTreeNode()
        self.current_size = 0
        self.min_p_i = 0
        
    def __len__(self):
        return self.current_size 
        
    def add_experience(self, experience, loss):
        if self.current_size >= self.max_size:    # Remove minimum p_i experience
            self.store.minimal_node().remove()
        self.store.attach_child(SumTreeNode(experience, pow(loss,self.omega)))
        self.min_p_i = self.store.minimal_node().p_i
        if self.current_size < self.max_size:
            self.current_size += 1
            
    def sample(self,batch_size):
        if batch_size > self.current_size:
            return None
        sample_keys = (np.random.rand(batch_size)*self.store.p_i).tolist()
        return [self.store.weighted_retrieve(p_samp) for p_samp in sample_keys]

In [66]:
# Unit test for PrioritizedReplayBuffer
test_rep_hyp = ReplayBufferHyperparameters(buffer_size = 8, omega = 1)
test_rep_buffer = PrioritizedReplayBuffer(test_rep_hyp)
test_experiences = ['one','two','three','four','five','six','seven','eight','nine']
test_weights = [0.99,2,3,1,2,3,1,2,3]
test_batch_size = 4
test_batches = 1000
for (e,wt) in zip(test_experiences,test_weights):
    test_rep_buffer.add_experience(e,wt)
test_samples = []
for i in range(test_batches):
    test_samples += [sample.data for sample in test_rep_buffer.sample(test_batch_size)]
print("Normalized Counts")
for exp in test_experiences:
    print(f'{exp}: {test_samples.count(exp) / test_samples.count("four")}')

Normalized Counts
one: 0.0
two: 2.384976525821596
three: 3.2300469483568075
four: 1.0
five: 2.187793427230047
six: 3.2629107981220655
seven: 1.215962441314554
eight: 2.291079812206573
nine: 3.2065727699530515


### Action Value Distribution Function (Neural Network)

The linear layers have noisy bias and weight components, implemented with factored gaussian noise according to [5].

In [ ]:
class NoisyLinear(nn.Module):
    def __init__(self,in_dim,out_dim,noise_init=0.5):
        super().__init__()
        self.deterministic_linear = nn.Linear(in_dim,out_dim)
        self.noisy_weights = \
            noise_init*torch.ones(in_dim,out_dim,dtype=torch.float64,requires_grad=True)
        self.noisy_bias = noise_init*torch.ones(1,out_dim,dtype=torch.float64,requires_grad=True)
    
    def noise_transform(self,x):
        # See section 3(b) of reference [5]
        return torch.mul(torch.sgn(x),torch.sqrt(torch.abs(x)))
    
    def forward(self,x):
        # Generate factorized gaussian noise
        in_dim_noise = self.noise_transform(torch.randn((self.noisy_weights.Size(dim=0),1)))
        out_dim_noise = self.noise_transform(torch.randn((1,self.noisy_weights.Size(dim=1))))
        
        weight_noise = torch.mul(self.noisy_weights,torch.matmul(in_dim_noise,out_dim_noise))
        bias_noise = torch.mul(self.noisy_bias,out_dim_noise)
        
        return self.deterministic_linear(x) + torch.matmul(weight_noise,x) + bias_noise

### Bellman Update (Loss) Computation

### Training Loop

#### Training Session Options
`max_episodes`: Maximum number of episodes before stopping training run<br>
`rewards_buffer`: Pass in empty list or existing list.  Average rewards per episode data is appended to this list<br>
`reward_window`: Length of averaging window for reporting rewards, in number of episodes<br>
`report_interval`: Interval in episodes for updating `rewards_buffer` and displaying status<br>
`solved_threshold`: Average reward over averaging window required to stop training early<br>

In [10]:
class TrainingOptions():
    def __init__(self,max_episodes=2000,
                 rewards_buffer=[],reward_window=100,
                 report_interval=50,solved_threshold=13):
        self.max_episodes = max_episodes
        self.rewards_buffer = rewards_buffer
        self.reward_window = reward_window
        self.report_interval = report_interval
        self.solved_threshold = solved_threshold

#### `train_net` Parameters
`net`: Parameterized model to use for training run<br>
`replay_buffer`: Replay buffer object to use for training run<br>
`optimization_hyperparameters`: `OptimizationHyperparameters` object to use for training run<br>
`train_options`: `TrainingOptions` object to use for training run

#### `train_net` Returns
Reference to the `rewards_buffer` of the supplied `train_options`

## Results

## References
[1] Hessel et. al., Rainbow: Combining Improvements in Deep Reinforcement Learning, arXiv:1710.02298 <br>
[2] Bellemare et. al., A Distributional Perspective on Reinforcement Learning, arXiv:1707.06887 <br>
[3] Schaul et. al., Prioritized Experience Replay, arXiv:1511.05952<br>
[4] http://www.sefidian.com/2022/09/09/sumtree-data-structure-for-prioritized-experience-replay-per-explained-with-python-code/<br>
[5] Fortunato et. al., Noisy Networks for Exploration, arXiv:1706.10295